In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Dependencies

In [ ]:
!pip install datasets
!pip install seqeval
!pip install transformers[torch]


!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 38.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=992d77a74c9c89bf4e26c23ae634b00afc9cf33ff43f8d45878915ea68359f11
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully bu

# Import Libraries

In [ ]:
import pandas as pd
import ast
import numpy as np
from collections import Counter
from transformers import DataCollatorForTokenClassification
import datasets
from datasets import Dataset
# visualization libraries
import matplotlib.pyplot as plt
import numpy as np

# pytorch libraries
import torch # the main pytorch library
import torch.nn as nn # the sub-library containing Softmax, Module and other useful functions
import torch.optim as optim # the sub-library containing the common optimizers (SGD, Adam, etc.)

# huggingface's transformers library
from transformers import RobertaForTokenClassification, RobertaTokenizer

# huggingface's datasets library
# from datasets import load_dataset

# the tqdm library used to show the iteration progress
import tqdm
tqdmn = tqdm.notebook.tqdm

from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_metric

# Utitlity Functions

In [ ]:
def remove_items(test_list, item):

    # using filter() + __ne__ to perform the task
    res = list(filter((item).__ne__, test_list))

    return res


def remove_punctuation(test, ttags):
  # initializing punctuations string
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for i, ele in enumerate(test):
    if ele in punc:
        # t = ele.replace(ele, "")
        # test[i] = t
        test.pop(i)
        ttags.pop(i)

  return test, ttags

def remove_element(test, ttags, element):
  # initializing punctuations string
  e = element
  for i, ele in enumerate(test):
    if ele == e:
        # t = ele.replace(ele, "")
        # test[i] = t
        test.pop(i)
        ttags.pop(i)

  return test, ttags

# Load all Folds

In [ ]:
# Create a list to store the file contents
train_tagged_tokens_list = []

for num in range(10):
    path = "/content/drive/MyDrive/NER/Urdu Name Entity Recognition Dataset (MK-PUCIT)/fold" + str(num+1) + "/urduDataTrainfold" + str(num+1) + ".txt"
    with open(path, "r", encoding="utf-8") as mytrainfile:
        train_tagged_tokens = mytrainfile.readlines()

    # Append the contents to the list
    train_tagged_tokens_list.append(train_tagged_tokens)
    print(f"Train Tagged Tokens: {len(train_tagged_tokens)} for fold: {num+1}")

# Now you have train_tagged_tokens_list[0], train_tagged_tokens_list[1], etc. for each file


Train Tagged Tokens: 673026 for fold: 1
Train Tagged Tokens: 674365 for fold: 2
Train Tagged Tokens: 674485 for fold: 3
Train Tagged Tokens: 674487 for fold: 4
Train Tagged Tokens: 674484 for fold: 5
Train Tagged Tokens: 674604 for fold: 6
Train Tagged Tokens: 674690 for fold: 7
Train Tagged Tokens: 674568 for fold: 8
Train Tagged Tokens: 673171 for fold: 9
Train Tagged Tokens: 672975 for fold: 10


In [ ]:
# Create a list to store the file contents
test_tagged_tokens_list = []

for num in range(10):
    path = "/content/drive/MyDrive/NER/Urdu Name Entity Recognition Dataset (MK-PUCIT)/fold" + str(num+1) + "/urduDataTestfold" + str(num+1) + ".txt"
    with open(path, "r", encoding="utf-8") as mytrainfile:
        test_tagged_tokens = mytrainfile.readlines()

    # Append the contents to the list
    test_tagged_tokens_list.append(test_tagged_tokens)
    print(f"Test Tagged Tokens: {len(test_tagged_tokens)} for fold: {num+1}")

# Now you have train_tagged_tokens_list[0], train_tagged_tokens_list[1], etc. for each file


Test Tagged Tokens: 289934 for fold: 1
Test Tagged Tokens: 288627 for fold: 2
Test Tagged Tokens: 288485 for fold: 3
Test Tagged Tokens: 288498 for fold: 4
Test Tagged Tokens: 288493 for fold: 5
Test Tagged Tokens: 288351 for fold: 6
Test Tagged Tokens: 288288 for fold: 7
Test Tagged Tokens: 288418 for fold: 8
Test Tagged Tokens: 289784 for fold: 9
Test Tagged Tokens: 289999 for fold: 10


# Check Which Fold has Most Tokens

In [ ]:
max = 0
max_fold = 0
for num in range(10):
   fold = test_tagged_tokens_list[num] + train_tagged_tokens_list[num]
   print(f"Length of Fold {num+1}: {len(fold)} As Set: {len(set(fold))}")
   if len(fold) > max:
      max = len(fold)
      max_fold = num+1
      print(f"Max Fold is: {max_fold} with {max} tokens")
   print("____________")

Length of Fold 1: 962960 As Set: 46696
Max Fold is: 1 with 962960 tokens
____________
Length of Fold 2: 962992 As Set: 46699
Max Fold is: 2 with 962992 tokens
____________
Length of Fold 3: 962970 As Set: 46699
____________
Length of Fold 4: 962985 As Set: 46698
____________
Length of Fold 5: 962977 As Set: 46698
____________
Length of Fold 6: 962955 As Set: 46697
____________
Length of Fold 7: 962978 As Set: 46699
____________
Length of Fold 8: 962986 As Set: 46698
____________
Length of Fold 9: 962955 As Set: 46697
____________
Length of Fold 10: 962974 As Set: 46700
____________


# SELECTING FOLD 2

In [ ]:
num = 1
fold2 = test_tagged_tokens_list[num] + train_tagged_tokens_list[num]

tagged_words = []
for tagged_token in fold2:
  tagged_token = tagged_token.replace('\t', ' ')
  tagged_token = tagged_token.replace('\n', '')
  # tagged_token = tagged_token.replace('\u202b', '')
  # tagged_token = tagged_token.replace(' ', '')
  tagged_words.append(tagged_token)

tagged_words = remove_items(tagged_words, '')

# Split the sentences and the tags
tokens, token_tags = [], []
for index, tagged_word in enumerate(tagged_words):
  # print(tagged_word)
  # print(f'index: {index}, tagged_word: {tagged_word}')
  word = tagged_word.split()
  # print(word)
  # print(f'index: {index}, token: {word[0]}')
  token = word[0]
  tag = word[1]
  # print(f'word[0]: {token}, word[1]: {tag}')
  tokens.append(token)
  token_tags.append(tag)


listed_tokens = [[token] for token in tokens]
listed_tags = [[token] for token in token_tags]
print('-----------------------------------------------')
print(tokens[0])
print("Train Tokens: ", format(len(tokens), ','))
print(listed_tokens[0])
print("Listed Train Tokens: ", format(len(listed_tokens), ','))
print(token_tags[0])
print("Train Tags: ", format(len(token_tags), ','))
print(listed_tags[0])
print("Listed Train Tags: ", format(len(listed_tags), ','))

-----------------------------------------------
مجھے
Train Tokens:  926,774
['مجھے']
Listed Train Tokens:  926,774
Other
Train Tags:  926,774
['Other']
Listed Train Tags:  926,774


In [ ]:
len(tagged_words), len(fold2)

(926774, 962992)

In [ ]:
from collections import Counter

# Get the count of unique values
value_counts = Counter(token_tags)

# Print the count of each unique value
print(value_counts)

Counter({'Other': 827056, 'Location': 43050, 'Person': 36186, 'Organization': 20482})


In [ ]:
36186+20482+43050

99718

In [ ]:
set(token_tags)

{'Location', 'Organization', 'Other', 'Person'}

# Data Pre-processing

### Tag Encoding

In [ ]:
tags = set(token_tags)
tags

{'Location', 'Organization', 'Other', 'Person'}

In [ ]:
for ind, tags in enumerate(token_tags):
  if token_tags[ind] == 'Others':
    token_tags[ind] = 'Other'

tags = set(token_tags)
tags

{'Location', 'Organization', 'Other', 'Person'}

In [ ]:
tag2index = {'Other': 0, 'Location': 1, 'Organization': 2, 'Person': 3}
index2tag = {0: 'Other', 1: 'Location', 2: 'Organization', 3: 'Person'}

In [ ]:
def get_tag_encoding(test_tags, tag2index):
  test_tags_y = []
  for tags in test_tags:
    test_tags_y.append(tag2index[tags])
      # test_tags_y.append([tag2index[t] for t in tags])
  return test_tags_y

tags_y = get_tag_encoding(token_tags, tag2index)

### Train/Test/Val Split
80% data for train, 10% each for test and val selected by multiplying the length of data  by 0.8 and the remaining data by 0.5

In [ ]:
# Create train, test, val splits: 80% train, 10% each for test and val
train_tags_y = tags_y[:741419]
print(len(train_tags_y))
test_tags_y = tags_y[741419:834198]
print(len(test_tags_y))
val_tags_y = tags_y[834198:]
print(len(val_tags_y))



train_tokens = tokens[:741419]
print(len(train_tokens))
test_tokens = tokens[741419:834198]
print(len(test_tokens))
val_tokens = tokens[834198:]
print(len(val_tokens))

741419
92779
92576
741419
92779
92576


In [ ]:
741419 + 92779 + 92576 == 926774

True

In [ ]:
import pandas as pd
train_df = pd.DataFrame({
     "tags" : train_tags_y,
     "tokens" : train_tokens
})

val_df = pd.DataFrame({
     "tags" : val_tags_y,
     "tokens" : val_tokens
})

test_df = pd.DataFrame({
     "tags" : test_tags_y,
     "tokens" : test_tokens
})


In [ ]:
train_df['tags'] = [[value] for value in list(train_df['tags'])]
train_df['tokens'] = [[value] for value in list(train_df['tokens'])]

val_df['tags'] = [[value] for value in list(val_df['tags'])]
val_df['tokens'] = [[value] for value in list(val_df['tokens'])]

test_df['tags'] = [[value] for value in list(test_df['tags'])]
test_df['tokens'] = [[value] for value in list(test_df['tokens'])]


In [ ]:
train_dataset = Dataset.from_dict(train_df)
val_dataset = Dataset.from_dict(val_df)
test_dataset = Dataset.from_dict(test_df)
dataset = datasets.DatasetDict({"train":train_dataset,"test":test_dataset, "validation":val_dataset})

# Compute Metrics

In [ ]:
label_list2= [[0], [1], [2], [3]]
# labels_list2 = [tensor(0), tensor(1), tensor(2), tensor(3)]
from evaluate import load

metric2 = load("seqeval")

# metric2 = load_metric("seqeval")
def compute_metrics2(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list2[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list2[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric2.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


# RoBERTa Finetuning

## Get RoBERTa Embeddings

In [ ]:
roberta_version = "urduhack/roberta-urdu-small"
tokenizer = RobertaTokenizer.from_pretrained(roberta_version)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

In [ ]:
def add_encodings(examples):

    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding='max_length', max_length=5)
    tokenized_inputs["labels"] = examples['tags']
    return tokenized_inputs

In [ ]:
dataset = dataset.map(add_encodings)
# dataset2 = dataset2.map(add_encodings, remove_columns=['tags', 'tokens'])

dataset.set_format(type='torch')
# dataset2.set_format(type='torch')

dataset
# dataset2

Map:   0%|          | 0/741419 [00:00<?, ? examples/s]

Map:   0%|          | 0/92779 [00:00<?, ? examples/s]

Map:   0%|          | 0/92576 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tags', 'tokens', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 741419
    })
    test: Dataset({
        features: ['tags', 'tokens', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 92779
    })
    validation: Dataset({
        features: ['tags', 'tokens', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 92576
    })
})

In [ ]:
from datasets import load_dataset

# Assuming you have a dataset named 'my_dataset'
# dataset.save_to_disk('/content/drive/MyDrive/NER/MK-PUCIT_fold10')
dataset.save_to_disk('/content/drive/MyDrive/NER/Tokenized_Data/MK-PUCIT_fold2_withTokens_RoBERTa')

Saving the dataset (0/1 shards):   0%|          | 0/741419 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/92779 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/92576 [00:00<?, ? examples/s]

## Initialize Model and Load Dataset

### Load Data

In [ ]:
from datasets import load_from_disk

# Load the dataset from the saved directory
dataset = load_from_disk('/content/drive/MyDrive/NER/Tokenized_Data/MK-PUCIT_fold2_withTokens_RoBERTa')
# {'Location': 0, 'Other': 1, 'Organization': 2, 'Person': 3},
#  {0: 'Location', 1: 'Other', 2: 'Organization', 3: 'Person'}

# You can now access the different splits (e.g., 'train', 'test', 'validation') using loaded_dataset['split_name']
train = dataset['train']
test = dataset['test']
validation = dataset['validation']


### Initialize Model

In [ ]:
roberta_version = "urduhack/roberta-urdu-small"
tokenizer = RobertaTokenizer.from_pretrained(roberta_version)

In [ ]:
# tag2index = {'Location': 0, 'Other': 1, 'Organization': 2, 'Person': 3}
# index2tag = {0: 'Location', 1: 'Other', 2: 'Organization', 3: 'Person'}
num_labels = len(tag2index)
print('num_labels: ', num_labels)

model = RobertaForTokenClassification.from_pretrained(roberta_version, num_labels=num_labels)

num_labels:  4


pytorch_model.bin:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at urduhack/roberta-urdu-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Data Collator

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

label_all_tokens = True
task = "ner"

## Set Training Arguments

In [ ]:
from transformers import EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

### Training Args 1_1 ###
training_args1_1 = TrainingArguments(
    output_dir="./fine_tune_bert_output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps = 1000,
    load_best_model_at_end=True,
    save_strategy='epoch'
)

### Trainer 1_1 ###
trainer1_1 = Trainer(
    model=model,
    args=training_args1_1,
    train_dataset=train,
    eval_dataset=validation,

    tokenizer=tokenizer,
    callbacks=[early_stopping],
    data_collator=data_collator,
    compute_metrics=compute_metrics2
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Train and Save

In [ ]:
trainer1_1.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.162000,0.177616,0.942793,0.942793,0.942793,0.942793
2,0.149500,0.174554,0.943646,0.943646,0.943646,0.943646
3,0.146100,0.178595,0.941443,0.941443,0.941443,0.941443


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [3] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [1] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [2] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: Use

#### Save model

In [ ]:
#### STATE DICT ####  (Recommended Way)
# Save Method
file_name = 'UNER_MK_PUCIT_fold2_trainer_statedict_RoBERTa.pth'
SAVE_PATH = '/content/drive/MyDrive/NER/Saved Models/RoBERTa/'
torch.save(model.state_dict(), SAVE_PATH+file_name)
# filename2 = 'Urdu_MK_PUCIT_DispropSampled_model1_statedict.pth'

In [ ]:
# Save the trained model
path = '/content/drive/MyDrive/NER/Saved Models/RoBERTa/UNER_MK_PUCIT_fold2_trainer_RoBERTa'
trainer1_1.save_model(output_dir=path)

In [ ]:
from transformers import AutoModelForSequenceClassification

# Fine-tune the model
# ...

# Save the trained model
model.save_pretrained('/content/drive/MyDrive/NER/Saved Models/RoBERTa/UNER_MK_PUCIT_fold2_RoBERTa')


## Test Model

In [ ]:
trainer1_1.evaluate

### Metrics

In [ ]:
predictions1, labels1, metrics1 = trainer1_1.predict(test, metric_key_prefix="predict")

In [ ]:
# Fold 2
'''
{'predict_loss': 0.18261802196502686,
 'predict_precision': 0.9419368761801996,
 'predict_recall': 0.9419368761801996,
 'predict_f1': 0.9419368761801996,
 'predict_accuracy': 0.9419368761801996,
 'predict_runtime': 60.5276,
 'predict_samples_per_second': 1531.121,
 'predict_steps_per_second': 23.94}
'''
metrics1

### Save Results

In [ ]:
predictions = np.argmax(predictions1, axis=2)

true_predictions12 = []
for prediction, label in zip(predictions, labels1):
    true_prediction = []
    for p, l in zip(prediction, label):
        if l != -100:
            # true_prediction.append(label_list2[p])
            true_predictions12.append(label_list2[p][0])

true_labels12 = []
for prediction, label in zip(predictions, labels1):
    true_prediction = []
    for p, l in zip(prediction, label):
        if l != -100:
            # true_prediction.append(label_list2[p])
            true_labels12.append(label_list2[l][0])


pred_df12 = pd.DataFrame({
    'true_labels': true_labels12,
    'true_predictions': true_predictions12,
    'token': test_tokens
})

pred_df12.head()

In [ ]:
pred_df12['pred_label'] = pred_df12['true_predictions'].map(index2tag)
pred_df12['true_label'] = pred_df12['true_labels'].map(index2tag)

pred_df12.head()

In [ ]:
pred_df12.to_csv('/content/drive/MyDrive/NER/UNER_RoBERTa_fold2_testResults.csv', index=False)

# MuRIL Finetuning

## Get MuRIL Embeddings

In [ ]:
from transformers import BertForTokenClassification, BertTokenizer
bert_version = "google/muril-base-cased"
tokenizer = BertTokenizer.from_pretrained(bert_version)

In [ ]:
def add_encodings(examples):

    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding='max_length', max_length=5)
    tokenized_inputs["labels"] = examples['tags']
    return tokenized_inputs

In [ ]:
dataset = dataset.map(add_encodings)
# dataset2 = dataset2.map(add_encodings, remove_columns=['tags', 'tokens'])

dataset.set_format(type='torch')
# dataset2.set_format(type='torch')

dataset
# dataset2

Map:   0%|          | 0/741419 [00:00<?, ? examples/s]

Map:   0%|          | 0/92779 [00:00<?, ? examples/s]

Map:   0%|          | 0/92576 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tags', 'tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 741419
    })
    test: Dataset({
        features: ['tags', 'tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 92779
    })
    validation: Dataset({
        features: ['tags', 'tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 92576
    })
})

In [ ]:
from datasets import load_dataset

# Assuming you have a dataset named 'my_dataset'
# dataset.save_to_disk('/content/drive/MyDrive/NER/MK-PUCIT_fold10')
dataset.save_to_disk('/content/drive/MyDrive/NER/Tokenized_Data/MK-PUCIT_fold2_withTokens_MuRIL')

Saving the dataset (0/1 shards):   0%|          | 0/741419 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/92779 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/92576 [00:00<?, ? examples/s]

## Initialize Model and Load Dataset

### Load Data

In [ ]:
from datasets import load_from_disk

# Load the dataset from the saved directory
# dataset = load_from_disk('/content/drive/MyDrive/NER/MK-PUCIT_fold10')
# {'Location': 0, 'Other': 1, 'Organization': 2, 'Person': 3},
#  {0: 'Location', 1: 'Other', 2: 'Organization', 3: 'Person'}

# You can now access the different splits (e.g., 'train', 'test', 'validation') using loaded_dataset['split_name']
train = dataset['train']
test = dataset['test']
validation = dataset['validation']


### Initialize Model

In [ ]:
bert_version = "google/muril-base-cased"
tokenizer = BertTokenizer.from_pretrained(bert_version)

In [ ]:
# tag2index = {'Location': 0, 'Other': 1, 'Organization': 2, 'Person': 3}
# index2tag = {0: 'Location', 1: 'Other', 2: 'Organization', 3: 'Person'}
num_labels = len(tag2index)
print('num_labels: ', num_labels)

model = BertForTokenClassification.from_pretrained(bert_version, num_labels=num_labels)

num_labels:  4


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Data Collator

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

label_all_tokens = True
task = "ner"

## Set Training Arguments

In [ ]:
from transformers import EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

### Training Args 1_1 ###
training_args1_1 = TrainingArguments(
    output_dir="./fine_tune_bert_output2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps = 1000,
    load_best_model_at_end=True,
    save_strategy='epoch'
)

### Trainer 1_1 ###
trainer1_1 = Trainer(
    model=model,
    args=training_args1_1,
    train_dataset=train,
    eval_dataset=validation,

    tokenizer=tokenizer,
    callbacks=[early_stopping],
    data_collator=data_collator,
    compute_metrics=compute_metrics2
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Train and Save

In [ ]:
trainer1_1.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.167800,0.185318,0.941443,0.941443,0.941443,0.941443
2,0.154500,0.185810,0.943085,0.943085,0.943085,0.943085
3,0.150000,0.180002,0.941162,0.941162,0.941162,0.941162
4,0.149300,0.193136,0.943236,0.943236,0.943236,0.943236
5,0.143400,0.189384,0.943538,0.943538,0.943538,0.943538
6,0.140800,0.196190,0.942977,0.942977,0.942977,0.942977


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [3] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [1] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [2] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: Use

TrainOutput(global_step=69510, training_loss=0.16504475507542754, metrics={'train_runtime': 10803.0415, 'train_samples_per_second': 686.306, 'train_steps_per_second': 10.724, 'total_flos': 1.135159471239192e+16, 'train_loss': 0.16504475507542754, 'epoch': 6.0})

#### Save model

In [ ]:
#### STATE DICT ####  (Recommended Way)
# Save Method
file_name = 'UNER_MK_PUCIT_fold2_trainer_statedict_MuRIL.pth'
SAVE_PATH = '/content/drive/MyDrive/NER/Saved Models/MuRIL/'
torch.save(model.state_dict(), SAVE_PATH+file_name)
# filename2 = 'Urdu_MK_PUCIT_DispropSampled_model1_statedict.pth'

## Test Model

In [ ]:
trainer1_1.evaluate

<bound method Trainer.evaluate of <transformers.trainer.Trainer object at 0x7a7bfd5fc850>>

### Metrics

In [ ]:
predictions1, labels1, metrics1 = trainer1_1.predict(test, metric_key_prefix="predict")

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [1] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [0] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [2] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: [3] seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [ ]:
# Fold 2
metrics1

{'predict_loss': 0.18199150264263153,
 'predict_precision': 0.9413768201856023,
 'predict_recall': 0.9413768201856023,
 'predict_f1': 0.9413768201856023,
 'predict_accuracy': 0.9413768201856023,
 'predict_runtime': 46.3432,
 'predict_samples_per_second': 2001.999,
 'predict_steps_per_second': 31.288}

### Save Results

In [ ]:
predictions = np.argmax(predictions1, axis=2)

true_predictions12 = []
for prediction, label in zip(predictions, labels1):
    true_prediction = []
    for p, l in zip(prediction, label):
        if l != -100:
            # true_prediction.append(label_list2[p])
            true_predictions12.append(label_list2[p][0])

true_labels12 = []
for prediction, label in zip(predictions, labels1):
    true_prediction = []
    for p, l in zip(prediction, label):
        if l != -100:
            # true_prediction.append(label_list2[p])
            true_labels12.append(label_list2[l][0])


pred_df12 = pd.DataFrame({
    'true_labels': true_labels12,
    'true_predictions': true_predictions12,
    'token': test_tokens
})

pred_df12.head()

,true_labels,true_predictions,token
0,1,1,ڈھاکہ
1,0,0,اور
2,1,1,الجیریا
3,0,0,سمیت
4,0,0,دیگر


In [ ]:
pred_df12['pred_label'] = pred_df12['true_predictions'].map(index2tag)
pred_df12['true_label'] = pred_df12['true_labels'].map(index2tag)

pred_df12.head()

,true_labels,true_predictions,token,pred_label,true_label
0,1,1,ڈھاکہ,Location,Location
1,0,0,اور,Other,Other
2,1,1,الجیریا,Location,Location
3,0,0,سمیت,Other,Other
4,0,0,دیگر,Other,Other


In [ ]:
pred_df12.to_csv('/content/drive/MyDrive/NER/UNER_MuRIL_fold2B_testResults.csv', index=False)